In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import re
import html as html_lib

# = = =  = = =  = = =

import json

In [2]:
directory_in = './file/2.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'2.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：92



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Fuel & Air,Fuel Tank Cap,GATES,31318,3548221C1;3548221C2,,"https://www.rockauto.com/en/parts/gates,31318,...","{""0"": ""https://www.rockauto.com/info/64/31318-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$104.79""}",5900,14824665,GATES;31318;5900;14824665
1,2,Fuel & Air,Fuel Tank Cap,GATES,31322,K221004,,"https://www.rockauto.com/en/parts/gates,31322,...","{""0"": ""https://www.rockauto.com/info/64/31322-...",https://www.rockauto.com/en/moreinfo.php?pk=59...,{},"{""0"": ""Category: Fuel Tank Cap (Only 1 Remaini...",,"{""0"": ""$62.79""}",5900,5937210,GATES;31322;5900;5937210
2,3,Fuel & Air,Fuel Tank Cap,GATES,31604,369495;4002059;4002060;4002303;4002305;4002571...,,"https://www.rockauto.com/en/parts/gates,31604,...","{""0"": ""https://www.rockauto.com/info/64/31604-...",https://www.rockauto.com/en/moreinfo.php?pk=10...,{},"{""0"": ""Category: Fuel Tank Cap (Only 1 Remaini...",,"{""0"": ""$3.51""}",5900,100338,GATES;31604;5900;100338
3,4,Fuel & Air,Fuel Tank Cap,GATES,31608,7031073;7031087,,"https://www.rockauto.com/en/parts/gates,31608,...","{""0"": ""https://www.rockauto.com/info/64/31608-...",https://www.rockauto.com/en/moreinfo.php?pk=10...,{},"{""0"": ""Category: Fuel Tank Cap [Wholesaler Clo...",,"{""0"": ""$5.79""}",5900,100339,GATES;31608;5900;100339
4,5,Fuel & Air,Fuel Tank Cap,GATES,31612,0092721020;12465217;125446;1254463;13219928;17...,,"https://www.rockauto.com/en/parts/gates,31612,...","{""0"": ""https://www.rockauto.com/info/64/31612-...",https://www.rockauto.com/en/moreinfo.php?pk=10...,{},"{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$5.31""}",5900,100343,GATES;31612;5900;100343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,Fuel & Air,Fuel Tank Cap,GATES,31857,30671264;30742058;30742326;31261589;312615890;...,,"https://www.rockauto.com/en/parts/gates,31857,...","{""0"": ""https://www.rockauto.com/info/64/31857-...",https://www.rockauto.com/en/moreinfo.php?pk=13...,{},"{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$14.69""}",5900,1390557,GATES;31857;5900;1390557
88,89,Fuel & Air,Fuel Tank Cap,GATES,31858,7034120,,"https://www.rockauto.com/en/parts/gates,31858,...","{""0"": ""https://www.rockauto.com/info/64/31858-...",https://www.rockauto.com/en/moreinfo.php?pk=38...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$9.84""}",5900,3860619,GATES;31858;5900;3860619
89,90,Fuel & Air,Fuel Tank Cap,GATES,31859,4584554AF;4584554AG;8U5A9C268AC;8U5Z9C268B;AE8...,,"https://www.rockauto.com/en/parts/gates,31859,...","{""0"": ""https://www.rockauto.com/info/64/31859-...",https://www.rockauto.com/en/moreinfo.php?pk=60...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$10.90""}",5900,6094077,GATES;31859;5900;6094077
90,91,Fuel & Air,Fuel Tank Cap,GATES,31860,7034121,,"https://www.rockauto.com/en/parts/gates,31860,...","{""0"": ""https://www.rockauto.com/info/64/31860-...",https://www.rockauto.com/en/moreinfo.php?pk=38...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$9.81""}",5900,3860664,GATES;31860;5900;3860664


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                  Fuel & Air
 Type                                                Fuel Tank Cap
 Manufacturer                                                GATES
 Part Number                                                 31318
 OEM                                           3548221C1;3548221C2
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/gates,31318,...
 Json_Src        {"0": "https://www.rockauto.com/info/64/31318-...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                     {"0": "Regular Locking Fuel Cap"}
 Json_Note 2                      {"0": "Category: Fuel Tank Cap"}
 Choose                                                           
 Json_Price                                       {"0": "$104.79"}
 Type Code                                        

In [5]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request = crawler_series['Info']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() == 'Continue Shopping':
                with open('./info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

print(soup.prettify())

1
2
3
<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
 <head>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="/jslib/combined/css/desktopmoreinfo.css?1733919856" rel="stylesheet" type="text/css"/>
  <script>
   if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift

In [6]:
dict_key, list_tr = {}, html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else []
for tr in list_tr:
    list_td = tr.xpath('./td')
    list_text = [etree.tostring(td).decode('utf-8') for td in list_td]
    list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if len(list_text) == 0:
        continue

    if list_text[0] in dict_key:
        dict_key[list_text[0]] += 1
    else:
        dict_key[list_text[0]] = 1

dict_specification, i, dict_count = {}, 0, {}
for i, tr in enumerate(list_tr):
    list_td = tr.xpath('./td')
    list_text = [etree.tostring(td).decode('utf-8') for td in list_td]
    list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if len(list_text) == 0:
        continue

    if dict_key[list_text[0]] == 1:
        dict_specification[str(i)] = {list_text[0]: 'N/A' if len(list_text) == 1 else list_text[1]}
    else:
        if list_text[0] in dict_count:
            dict_count[list_text[0]] += 1
        else:
            dict_count[list_text[0]] = 1
        dict_specification[str(i)] = {f'{list_text[0]}-{dict_count[list_text[0]]}': 'N/A' if len(list_text) == 1 else list_text[1]}

    i += 1

dict_specification

{'0': {'Camshaft Sprocket Material': 'High Alloy Steel'},
 '1': {'Keyway Adjustable': 'No'},
 '2': {'Sprockets Included': 'Yes'},
 '3': {'Timing Chain Type': 'Silent'}}

In [7]:
dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in enumerate(list_tr):
    dict_kit[str(i)], list_td = {}, tr.xpath('./td')

    dict_kit[str(i)]['[Kit] Quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    # = = =  = = =  = = =

    dict_kit[str(i)]['[Kit] Type'] = list_td[1].xpath('./text()')[0].strip()

    # = = =  = = =  = = =

    dict_ = json.loads(tr.xpath('./td[contains(@name, "listing_data_essential[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Type Code'] = dict_['parttype']
    dict_kit[str(i)]['[Kit] Part Code'] = dict_['partkey']

    # = = =  = = =  = = =

    dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Part Number'] = dict_2['partnumber']
    dict_kit[str(i)]['[Kit] Manufacturer'] = dict_2['catalogname']

    # = = =  = = =  = = =

    dict_kit[str(i)]['[Kit] Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['[Kit] Manufacturer']},{dict_kit[str(i)]['[Kit] Part Number']},{dict_kit[str(i)]['[Kit] Type']},{dict_kit[str(i)]['[Kit] Type Code']}'''

    # = = =  = = =  = = =

    list_a = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
    info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
    dict_kit[str(i)]['[Kit] Info'] = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['[Kit] Part Code']}&pt={dict_kit[str(i)]['[Kit] Type Code']}&Lennon=1'''

    # = = =  = = =  = = =

    list_input = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
    list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

    dict_src, j = {}, 0
    for dict_ in list_dict:
        list_src = dict_.get('Slots', [])
        for src in list_src:
            dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
            j += 1

    dict_kit[str(i)]['[Kit] Json_Src'] = json.dumps(dict_src)

    # = = =  = = =  = = =

    dict_kit[str(i)]['[Kit] JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['[Kit] Manufacturer']};{dict_kit[str(i)]['[Kit] Part Number']};{dict_kit[str(i)]['[Kit] Type Code']};{dict_kit[str(i)]['[Kit] Part Code']}'''

dict_kit

{'0': {'[Kit] Quantity': 1,
  '[Kit] Type': 'Timing Chain',
  '[Kit] Type Code': '5724',
  '[Kit] Part Code': '8306448',
  '[Kit] Part Number': '715F',
  '[Kit] Manufacturer': 'MELLING',
  '[Kit] Url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724',
  '[Kit] Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
  '[Kit] Json_Src': '{"0": "https://www.rockauto.com/info/583/715F-1-ANG.jpg"}',
  '[Kit] JOIN_MPNTCPC': 'MELLING;715F;5724;8306448'},
 '1': {'[Kit] Quantity': 1,
  '[Kit] Type': 'Timing Chain Guide / Damper',
  '[Kit] Type Code': '5728',
  '[Kit] Part Code': '8307424',
  '[Kit] Part Number': 'BG5515',
  '[Kit] Manufacturer': 'MELLING',
  '[Kit] Url': 'https://www.rockauto.com/en/parts/MELLING,BG5515,Timing Chain Guide / Damper,5728',
  '[Kit] Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728',
  '[Kit] Json_Src': '{"0": "https://www.rockauto.com/info/583/BG5515-1-SID.jpg"}',
  '[Kit] JOIN_MPNTCPC': 'MELLING;BG5515;

In [8]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Group': crawler_series['Group'],
                          'Type': crawler_series['Type'],
                          'Manufacturer': crawler_series['Manufacturer'],
                          'Part Number': crawler_series['Part Number'],
                          'OEM': crawler_series['OEM'],
                          'Picture': crawler_series['Picture'],
                          'Url': crawler_series['Url'],
                          'Json_Src': crawler_series['Json_Src'],
                          'Info': crawler_series['Info'],
                          'Json_Note 1': crawler_series['Json_Note 1'],
                          'Json_Note 2': crawler_series['Json_Note 2'],
                          'Choose': crawler_series['Choose'],
                          'Json_Price': crawler_series['Json_Price'],
                          'Json_Specification': json.dumps(dict_specification),
                          'Json_Kit': json.dumps(dict_kit),
                          'Type Code': crawler_series['Type Code'],
                          'Part Code': crawler_series['Part Code'],
                          'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC']})

list_dict_correct

[{'No': 1,
  'Group': 'Fuel & Air',
  'Type': 'Fuel Tank Cap',
  'Manufacturer': 'GATES',
  'Part Number': '31318',
  'OEM': '3548221C1;3548221C2',
  'Picture': '',
  'Url': 'https://www.rockauto.com/en/parts/gates,31318,fuel+tank+cap,5900',
  'Json_Src': '{"0": "https://www.rockauto.com/info/64/31318-1_ANG_A.jpg", "1": "https://www.rockauto.com/info/64/31318-2_TOP_A.jpg", "2": "https://www.rockauto.com/info/64/31318-3_BOT_A.jpg"}',
  'Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756&jsn=3',
  'Json_Note 1': '{"0": "Regular Locking Fuel Cap"}',
  'Json_Note 2': '{"0": "Category: Fuel Tank Cap"}',
  'Choose': '',
  'Json_Price': '{"0": "$104.79"}',
  'Json_Specification': '{"0": {"Camshaft Sprocket Material": "High Alloy Steel"}, "1": {"Keyway Adjustable": "No"}, "2": {"Sprockets Included": "Yes"}, "3": {"Timing Chain Type": "Silent"}}',
  'Json_Kit': '{"0": {"[Kit] Quantity": 1, "[Kit] Type": "Timing Chain", "[Kit] Type Code": "5724", "[Kit] Part Code": "8306448

In [9]:
crawler_status = True

crawler_status

True

In [10]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                  Fuel & Air
 Type                                                Fuel Tank Cap
 Manufacturer                                                GATES
 Part Number                                                 31318
 OEM                                           3548221C1;3548221C2
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/gates,31318,...
 Json_Src        {"0": "https://www.rockauto.com/info/64/31318-...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                     {"0": "Regular Locking Fuel Cap"}
 Json_Note 2                      {"0": "Category: Fuel Tank Cap"}
 Choose                                                           
 Json_Price                                       {"0": "$104.79"}
 Type Code                                                    

In [11]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_info.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Json_Specification,Json_Kit,Type Code,Part Code,JOIN_MPNTCPC
0,1,Fuel & Air,Fuel Tank Cap,GATES,31318,3548221C1;3548221C2,,"https://www.rockauto.com/en/parts/gates,31318,...","{""0"": ""https://www.rockauto.com/info/64/31318-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$104.79""}","{""0"": {""Camshaft Sprocket Material"": ""High All...","{""0"": {""[Kit] Quantity"": 1, ""[Kit] Type"": ""Tim...",5900,14824665,GATES;31318;5900;14824665


In [12]:
print('Output Finished !')
print()

Output Finished !



In [13]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_info_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC,Request_Url
0,1,Fuel & Air,Fuel Tank Cap,GATES,31318,3548221C1;3548221C2,,"https://www.rockauto.com/en/parts/gates,31318,...","{""0"": ""https://www.rockauto.com/info/64/31318-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""Regular Locking Fuel Cap""}","{""0"": ""Category: Fuel Tank Cap""}",,"{""0"": ""$104.79""}",5900,14824665,GATES;31318;5900;14824665,https://www.rockauto.com/en/moreinfo.php?pk=83...


In [14]:
print('Done ~')

Done ~
